<a href="https://colab.research.google.com/github/Magic-Freeza/predi-o_licita-o_compras_gov/blob/main/Predi%C3%A7%C3%A3o_de_pre%C3%A7os_de_licita%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas importadas

In [ ]:
# !pip install ydata-profiling

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.9/390.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.8/687.8 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 82.7 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=97076d0dbfdc83ba9f7f6700da08d2f556dc8162d668a6df74ef0c8990cb0745
  Stored in directory: /root/.cache/pip/wheels/8d/55/1a/19cd535375ed1ede0c996405ebffe34b196d78e2d9545723a2
Successfully built htmlmin


In [ ]:
!pip install catboost

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
# from ydata_profiling import ProfileReport
from google.colab import files

from google.colab import drive
import os
from concurrent.futures import ThreadPoolExecutor


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import scale, StandardScaler, OneHotEncoder, OrdinalEncoder


import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.inspection import partial_dependence
from scipy.optimize import minimize
from itertools import product

from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor

# codigo_serviço_list: definição dos codigos de serviços alvos da empresa

Não precisa ser executada sempre, já temos os valores de codigo de serviço de interresse da empresa.

In [ ]:
def consultar_item_servico(
    pagina=None,
    codigoSecao=None,
    codigoDivisao=None,
    codigoGrupo=None,
    codigoClasse=None,
    codigoSubclasse=None,
    codigoCpc=None,
    codigoServico=None,
    exclusivoCentralCompras=None,
    statusServico=None,
    tamanhoPagina=None,
):

    url = "https://dadosabertos.compras.gov.br/modulo-servico/6_consultarItemServico"
    # Configurar os parâmetros da API
    params = {
        "pagina": pagina,
        "codigoSecao": codigoSecao,
        "codigoDivisao": codigoDivisao,
        "codigoGrupo": codigoGrupo,
        "codigoClasse": codigoClasse,
        "codigoSubclasse": codigoSubclasse,
        "codigoCpc": codigoCpc,
        "codigoServico": codigoServico,
        "exclusivoCentralCompras": exclusivoCentralCompras,
        "statusServico": statusServico,
        "tamanhoPagina": tamanhoPagina,
    }

    # Remove parâmetros com valor None
    params = {k: v for k, v in params.items() if v is not None}

    try:
      response = requests.get(url, params=params)
      response.raise_for_status()  # Levanta erro para respostas com status != 200
      return response.json()  # Retorna a resposta em formato JSON
    except requests.exceptions.RequestException as e:
      print(f"Erro ao consultar a API: {e}")
      return None

- 'grupos_interesse_tic_geral' corresponde a todos os serviços de TIC solicitados pelo compras.gov.

- 'grupos_interesse' correspnde aos serviços de interresse da youtan.

In [ ]:
grupos_interesse_tic_geral = [
    111, 112, 113, 114, 115, 116, 117, 131, 141, 142,
    151, 152, 153, 161, 162, 163, 164, 165, 166, 167,
    168, 171, 172, 173, 174] # (codigoGrupo) Informações coletadas a partir da analise do compras_gov_lista_grupo_servico.csv

grupos_interesse = [111, 112, 114, 115, 116, 117, 163, 164, 171, 172, 173]


In [ ]:
print(len(grupos_interesse))

11


In [ ]:
pagina = 1
codigoSecao = None
codigoDivisao = None
codigoGrupo = None
codigoClasse = None
codigoSubclasse = None
codigoCpc = None
codigoServico = None
exclusivoCentralCompras = None
statusServico = None
tamanhoPagina = 500

In [ ]:
# Consulta inicial
df_item_servico = pd.DataFrame()

total_registros_item_servico_list = list()
paginas_restantes_item_servico_list = list()
grupos_interesse_list = [111, 112, 114, 115, 116, 117, 163, 164, 171, 172, 173]

for n in  grupos_interesse:

  df_item_servico_sup = pd.DataFrame()
  codigoGrupo = n

  dados_item_servico =  consultar_item_servico(
      pagina,
      codigoSecao,
      codigoDivisao,
      codigoGrupo,
      codigoClasse,
      codigoSubclasse,
      codigoCpc,
      codigoServico,
      exclusivoCentralCompras,
      statusServico,
      tamanhoPagina)

  # Processa os resultados
  if dados_item_servico:
    resultados_item_servico = dados_item_servico.get("resultado", [])
    total_registros_item_servico = dados_item_servico.get("totalRegistros", 0)
    paginas_restantes_item_servico = dados_item_servico.get("paginasRestantes", 0)

    #print(f"Total de registros: {total_registros_item_servico}")
    #print(f"Total de páginas restantes: {paginas_restantes_item_servico}")

    total_registros_item_servico_list.append(total_registros_item_servico)
    paginas_restantes_item_servico_list.append(paginas_restantes_item_servico)

  else:
    print("Erro ao obter dados da API.")

  df_item_servico_sup = pd.DataFrame(resultados_item_servico)

  df_item_servico = pd.concat([df_item_servico, df_item_servico_sup],ignore_index=True)

In [ ]:
df_info_grupo_servico = pd.DataFrame({
    'codigoGrupo': grupos_interesse_list,
    'totalRegistros': total_registros_item_servico_list,
    'paginasRestantes': paginas_restantes_item_servico_list
})

df_info_grupo_servico.head(25)

,codigoGrupo,totalRegistros,paginasRestantes
0,111,14,0
1,112,2,0
2,114,1,0
3,115,1,0
4,116,1,0
5,117,1,0
6,163,6,0
7,164,1,0
8,171,4,0
9,172,2,0


In [ ]:
df_info_grupo_servico['totalRegistros'].sum()

39

In [ ]:
df_item_servico.head()

,codigoSecao,nomeSecao,codigoDivisao,nomeDivisao,codigoGrupo,nomeGrupo,codigoClasse,nomeClasse,codigoSubclasse,nomeSubclasse,codigoServico,nomeServico,codigoCpc,exclusivoCentralCompras,statusServico,dataHoraAtualizacao
0,1,SERVIÇOS DE TECNOLOGIA DA INFORMAÇÃO E COMUNIC...,11,"SERVIÇOS DE DESENVOLVIMENTO, MANUTENÇÃO E ...",111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,1111,SERVIÇOS DE DESENVOLVIMENTO DE SOFTWARE E OU ...,None,None,25852,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,1111,False,True,2022-08-24T09:44:36.082254
1,1,SERVIÇOS DE TECNOLOGIA DA INFORMAÇÃO E COMUNIC...,11,"SERVIÇOS DE DESENVOLVIMENTO, MANUTENÇÃO E ...",111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,1111,SERVIÇOS DE DESENVOLVIMENTO DE SOFTWARE E OU ...,None,None,25860,DESENVOLVIMENTO DE NOVO SOFTWARE - PHP,1111,False,True,2021-10-16T09:09:59.689615
2,1,SERVIÇOS DE TECNOLOGIA DA INFORMAÇÃO E COMUNIC...,11,"SERVIÇOS DE DESENVOLVIMENTO, MANUTENÇÃO E ...",111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,1111,SERVIÇOS DE DESENVOLVIMENTO DE SOFTWARE E OU ...,None,None,25879,DESENVOLVIMENTO DE NOVO SOFTWARE- FRAMEWORK .N...,1111,False,True,2021-10-16T09:09:59.689615
3,1,SERVIÇOS DE TECNOLOGIA DA INFORMAÇÃO E COMUNIC...,11,"SERVIÇOS DE DESENVOLVIMENTO, MANUTENÇÃO E ...",111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,1111,SERVIÇOS DE DESENVOLVIMENTO DE SOFTWARE E OU ...,None,None,25887,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PY...,1111,False,True,2022-08-24T09:44:36.085229
4,1,SERVIÇOS DE TECNOLOGIA DA INFORMAÇÃO E COMUNIC...,11,"SERVIÇOS DE DESENVOLVIMENTO, MANUTENÇÃO E ...",111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,1111,SERVIÇOS DE DESENVOLVIMENTO DE SOFTWARE E OU ...,None,None,25895,DESENVOLVIMENTO DE NOVO SOFTWARE - DISPOSITIVO...,1111,False,True,2021-10-16T09:09:59.689615


In [ ]:
df_item_servico = df_item_servico.drop(columns=['codigoSecao',	'nomeSecao',	'codigoDivisao',	'nomeDivisao',	'codigoClasse',	'nomeClasse',	'codigoSubclasse',	'nomeSubclasse', 'codigoCpc',	'exclusivoCentralCompras',	'statusServico',	'dataHoraAtualizacao'])
df_item_servico.head()

,codigoGrupo,nomeGrupo,codigoServico,nomeServico
0,111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,25852,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA
1,111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,25860,DESENVOLVIMENTO DE NOVO SOFTWARE - PHP
2,111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,25879,DESENVOLVIMENTO DE NOVO SOFTWARE- FRAMEWORK .N...
3,111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,25887,DESENVOLVIMENTO E OU EVOLUCAO DE SOFTWATE - PY...
4,111,SERVIÇOS DE DESENVOLVIMENTO E MANUTENÇÃO DE S...,25895,DESENVOLVIMENTO DE NOVO SOFTWARE - DISPOSITIVO...


In [ ]:
codigo_serviço_list = list(df_item_servico['codigoServico'].unique())
print(codigo_serviço_list)
print(len(codigo_serviço_list))

[25852, 25860, 25879, 25887, 25895, 25909, 25917, 25925, 25933, 25941, 25950, 25968, 25976, 25984, 25992, 26000, 26026, 26034, 26042, 30001, 27030, 27049, 27057, 27065, 27073, 27588, 27081, 27294, 27308, 27316, 30178, 27324, 27901, 833, 21148, 21644, 21652, 27332, 27340]
39


Esta planilha deve ser gerada para confirmar os grupos de interresse quando necessario

In [ ]:
'''
df_item_servico.to_csv("item_servico.csv", index=False)
files.download("item_servico.csv")
'''

'\ndf_item_servico.to_csv("item_servico.csv", index=False)\nfiles.download("item_servico.csv")\n'

# df: preços dos ganhadores das licitações por itens

In [ ]:
def consultar_preco_servico(
    pagina = None,
    codigoItemCatalogo = None):

  url = 'https://dadosabertos.compras.gov.br/modulo-pesquisa-preco/3_consultarServico'

    # Parâmetros da consulta
  params = {
    'pagina': pagina,
    'codigoItemCatalogo': codigoItemCatalogo
  }

    # Remove parâmetros com valor None
  params = {k: v for k, v in params.items() if v is not None}

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Levanta erro para respostas com status != 200
    return response.json()  # Retorna a resposta em formato JSON
  except requests.exceptions.RequestException as e:
    print(f"Erro ao consultar a API: {e}")
    return None

In [ ]:
# Parâmetros da consulta
pagina = 1
codigoItemCatalogo = None

In [ ]:
codigo_serviço_list = [25852, 25860, 25879, 25887, 25895, 25909, 25917, 25925, 25933, 25941, 25950, 25968, 25976, 25984, 25992, 26000, 26026, 26034, 26042, 30001, 27030, 27049, 27057, 27065, 27073, 27588, 27081, 27294, 27308, 27316, 30178, 27324, 27901, 833, 21148, 21644, 21652, 27332, 27340]

In [ ]:
total_registros_preco_servico_list = []
paginas_restantes_preco_servico_list = []
codigo_servico_list_apoio = []
dados_principal_dict = []

def processar_codigo_servico(codigoItemCatalogo):
    pagina = None
    dados_coletados = []

    # Consulta inicial
    dados_principal = consultar_preco_servico(pagina, codigoItemCatalogo)

    if not dados_principal:
        print(f"Erro ao obter dados da API para código {codigoItemCatalogo}.")
        return None

    dados_coletados.extend(dados_principal.get("resultado", []))
    total_registros = dados_principal.get("totalRegistros", 0)
    paginas_restantes = dados_principal.get("paginasRestantes", 0)

    pagina = 1
    while paginas_restantes > 0:
        pagina += 1
        dados_principal = consultar_preco_servico(pagina, codigoItemCatalogo)

        if dados_principal:
            dados_coletados.extend(dados_principal.get("resultado", []))
            paginas_restantes = dados_principal.get("paginasRestantes", 0)
        else:
            print(f"Erro ao obter página {pagina} para código {codigoItemCatalogo}.")
            break

    return {
        "resultado": dados_coletados,
        "totalRegistros": total_registros,
        "paginasRestantes": paginas_restantes,
        "codigoItemCatalogo": codigoItemCatalogo
    }


In [ ]:
# Processamento paralelo
with ThreadPoolExecutor(max_workers=8) as executor:
    resultados = list(executor.map(processar_codigo_servico, codigo_serviço_list))

# Filtrar resultados válidos e organizar os dados
for res in filter(None, resultados):
    dados_principal_dict.extend(res["resultado"])
    total_registros_preco_servico_list.append(res["totalRegistros"])
    paginas_restantes_preco_servico_list.append(res["paginasRestantes"])
    codigo_servico_list_apoio.append(res["codigoItemCatalogo"])


In [ ]:
df = pd.DataFrame(dados_principal_dict)

In [ ]:
df.duplicated().sum()

441

In [ ]:
df = df.drop_duplicates()
# df.info()

In [ ]:
df = df.drop(columns= [
    'forma',
    'poder',
    'esfera'
    ])

# df.info()

In [ ]:
# Transformar colunas no formato de data
colunas_para_data = [
    'dataCompra', 'dataResultado','dataHoraAtualizacaoUasg','dataHoraAtualizacaoCompra',
    'dataHoraAtualizacaoItem' ]

for coluna in colunas_para_data:
    df[coluna] = pd.to_datetime(df[coluna], errors='coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6149 entries, 0 to 6589
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   idCompra                   6149 non-null   object             
 1   idItemCompra               6149 non-null   int64              
 2   modalidade                 6149 non-null   int64              
 3   criterioJulgamento         6149 non-null   object             
 4   numeroItemCompra           6149 non-null   int64              
 5   descricaoItem              6149 non-null   object             
 6   codigoItemCatalogo         6149 non-null   int64              
 7   nomeUnidadeMedida          5574 non-null   object             
 8   siglaUnidadeMedida         6147 non-null   object             
 9   quantidade                 6149 non-null   float64            
 10  precoUnitario              6149 non-null   float64            
 11  percentua

In [ ]:
df.head()

,idCompra,idItemCompra,modalidade,criterioJulgamento,numeroItemCompra,descricaoItem,codigoItemCatalogo,nomeUnidadeMedida,siglaUnidadeMedida,quantidade,...,codigoMunicipio,municipio,estado,codigoOrgao,nomeOrgao,dataCompra,dataHoraAtualizacaoCompra,dataHoraAtualizacaoItem,dataResultado,dataHoraAtualizacaoUasg
0,98122305000072022,787350,5,V,1,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,25852,PONTO DE FUNÇÃO,PF,1.0,...,2211209,URUÇUÍ,PI,94220,ESTADO DO PIAUI,2022-05-10 03:00:00+00:00,2023-05-11 03:30:00.049171,2023-05-11 04:00:00.049623,2022-05-10 03:00:00+00:00,2019-12-15 14:02:00
1,29000205000822022,1662699,5,V,1,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,25852,UND SERVIÇO TÉCNICO,UST,10.0,...,5300108,BRASÍLIA,DF,29101,DEFENSORIA PUBLICA DA UNIAO,2022-10-24 03:00:00+00:00,2023-10-25 03:30:00.001894,2023-10-25 04:00:00.005130,2022-10-24 03:00:00+00:00,2024-06-17 15:36:00
2,98755305000372023,3520859,5,V,16,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,25852,UND SERVIÇO TÉCNICO,UST,12.0,...,4107801,FLORAÍ,PR,97996,PREFEITURA DE FLORAI - PR,2023-09-05 03:00:00+00:00,2024-09-06 03:29:59.120669,2024-09-06 03:59:59.105024,2023-09-05 03:00:00+00:00,2023-06-26 16:38:00
3,98755305000372023,3520864,5,V,21,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,25852,UND SERVIÇO TÉCNICO,UST,12.0,...,4107801,FLORAÍ,PR,97996,PREFEITURA DE FLORAI - PR,2023-09-05 03:00:00+00:00,2024-09-06 03:29:59.120669,2024-09-06 03:59:59.105024,2023-09-05 03:00:00+00:00,2023-06-26 16:38:00
4,98755305000372023,3520865,5,V,22,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,25852,UND SERVIÇO TÉCNICO,UST,12.0,...,4107801,FLORAÍ,PR,97996,PREFEITURA DE FLORAI - PR,2023-09-05 03:00:00+00:00,2024-09-06 03:29:59.120669,2024-09-06 03:59:59.105024,2023-09-05 03:00:00+00:00,2023-06-26 16:38:00


# df_licitacoes_idCompra: contratações por id gerados no df principal para balizamento dos preços estimados



In [ ]:
def consultar_licitacao_id(
    idCompra,dataAtualizacaoPncp):

  url = 'https://dadosabertos.compras.gov.br/modulo-contratacoes/1.1_consultarContratacoes_PNCP_14133_Id'

    # Parâmetros da consulta
  params = {
    'idCompra': idCompra,
    'dataAtualizacaoPncp': dataAtualizacaoPncp
  }

    # Remove parâmetros com valor None
  params = {k: v for k, v in params.items() if v is not None}

  try:
    response = requests.get(url, params=params)
    response.raise_for_status()  # Levanta erro para respostas com status != 200
    return response.json()  # Retorna a resposta em formato JSON
  except requests.exceptions.RequestException as e:
    print(f"Erro ao consultar a API: {e}")
    return None

In [ ]:
# definindo variaveis
idCompra = None
dataAtualizacaoPncp = None

In [ ]:
# idCompra_list = dict(df['idCompra','dataHoraAtualizacaoUasg'].unique())

df_apoio_dicionario_compras = df[['idCompra', 'dataHoraAtualizacaoUasg']]

df_apoio_dicionario_compras['dataHoraAtualizacaoUasg'] = pd.to_datetime(df_apoio_dicionario_compras['dataHoraAtualizacaoUasg']).dt.date.astype(str)

df_apoio_dicionario_compras = df_apoio_dicionario_compras.drop_duplicates()

df_apoio_dicionario_compras['idCompra'] = df_apoio_dicionario_compras['idCompra'].astype(int)

dicionarios_compras = df_apoio_dicionario_compras[['idCompra', 'dataHoraAtualizacaoUasg']].to_dict(orient='records')

dicionarios_compras_len = len(dicionarios_compras)


<ipython-input-37-3ad00d74e851>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_apoio_dicionario_compras['dataHoraAtualizacaoUasg'] = pd.to_datetime(df_apoio_dicionario_compras['dataHoraAtualizacaoUasg']).dt.date.astype(str)


In [ ]:
total_registros_licitacoes_id_list = []
paginas_restantes_licitacoes_id_list = []
idCompra_list_apoio = []
dataAtualizacaoPncp_list_apoio = []
dados_licitacoes_idCompra_dict = []

def processar_dicionario(dicionario_compras):
    resultado = consultar_licitacao_id(dicionario_compras['idCompra'], dicionario_compras['dataHoraAtualizacaoUasg'])
    if resultado:
        return {
            "resultado": resultado.get("resultado", []),
            "totalRegistros": resultado.get("totalRegistros", 0),
            "paginasRestantes": resultado.get("paginasRestantes", 0),
            "idCompra": dicionario_compras['idCompra'],
            "dataAtualizacao": dicionario_compras['dataHoraAtualizacaoUasg'],
        }
    return None

# Processamento paralelo
with ThreadPoolExecutor(max_workers=8) as executor:
    resultados = list(executor.map(processar_dicionario, dicionarios_compras))

# Filtrar resultados não nulos e organizar dados
for res in filter(None, resultados):
    dados_licitacoes_idCompra_dict.extend(res["resultado"])
    total_registros_licitacoes_id_list.append(res["totalRegistros"])
    paginas_restantes_licitacoes_id_list.append(res["paginasRestantes"])
    idCompra_list_apoio.append(res["idCompra"])
    dataAtualizacaoPncp_list_apoio.append(res["dataAtualizacao"])

In [ ]:
df_licitacoes_idCompra = pd.DataFrame(dados_licitacoes_idCompra_dict)
df_licitacoes_idCompra.head()

,idCompra,numeroControlePNCP,anoCompraPncp,sequencialCompraPncp,orgaoEntidadeCnpj,orgaoSubrogadoCnpj,codigoOrgao,orgaoEntidadeRazaoSocial,orgaoSubrogadoRazaoSocial,orgaoEntidadeEsferaId,...,tipoInstrumentoConvocatorioNome,modoDisputaNomePncp,valorTotalEstimado,valorTotalHomologado,dataInclusaoPncp,dataAualizacaoPncp,dataPublicacaoPncp,dataAberturaPropostaPncp,dataEncerramentoPropostaPncp,contratacaoExcluida
0,98767705900122024,76416940000128-1-000433/2024,2024,433,76416940000128,None,39224,ESTADO DO PARANA,None,E,...,Edital,Aberto,81066.530,63951.480,2024-03-12T07:07:17,2024-03-12T07:07:17,2024-03-12T07:07:17,2024-03-12T08:00:00,2024-04-01T09:00:00,False
1,92604006900912023,07248660000135-1-000003/2023,2023,3,07248660000135,None,57107,DEFENSORIA PUBLICA DO ESTADO DO TOCANTINS,None,E,...,Ato que autoriza a Contratação Direta,Não se aplica,24924.500,24924.500,2023-12-19T10:58:41,2023-12-19T10:58:41,2023-12-19T10:58:41,None,None,False
2,98288705900222024,12333746000104-1-000072/2024,2024,72,12333746000104,None,63481,MUNICIPIO DE VICOSA,None,M,...,Edital,Aberto-Fechado,1046037.600,711600.000,2024-10-18T07:42:40,2024-10-18T07:42:40,2024-10-18T07:42:40,2024-10-18T08:00:00,2024-10-31T08:30:00,False
3,92547307000062025,12395125000147-1-000002/2025,2025,2,12395125000147,None,9817,ALAGOAS TRIBUNAL DE CONTAS DO ESTADO DE ALAGOAS,None,E,...,Ato que autoriza a Contratação Direta,Não se aplica,6860000.004,6860000.004,2024-12-26T12:22:40,2024-12-26T12:22:40,2024-12-26T12:22:40,None,None,False
4,97400405900312024,26963645000113-1-000062/2024,2024,62,26963645000113,None,23505,CAMARA LEGISLATIVA DO DISTRITO FEDERAL,None,D,...,Edital,Aberto,4723671.490,3542753.760,2024-08-30T07:44:28,2024-08-30T07:44:28,2024-08-30T07:44:28,2024-08-30T09:00:00,2024-09-17T09:30:00,False


In [ ]:
'''df.to_csv("compras_gov_preços_licitações.csv", index=False)
files.download("compras_gov_preços_licitações.csv")'''

'df.to_csv("compras_gov_preços_licitações.csv", index=False)\nfiles.download("compras_gov_preços_licitações.csv")'

In [ ]:
df_licitacoes_idCompra.duplicated().sum()

0

In [ ]:
df_licitacoes_idCompra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 52 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   idCompra                               1068 non-null   object 
 1   numeroControlePNCP                     1068 non-null   object 
 2   anoCompraPncp                          1068 non-null   int64  
 3   sequencialCompraPncp                   1068 non-null   int64  
 4   orgaoEntidadeCnpj                      1068 non-null   object 
 5   orgaoSubrogadoCnpj                     1 non-null      object 
 6   codigoOrgao                            1068 non-null   int64  
 7   orgaoEntidadeRazaoSocial               1068 non-null   object 
 8   orgaoSubrogadoRazaoSocial              1 non-null      object 
 9   orgaoEntidadeEsferaId                  1068 non-null   object 
 10  orgaoSubrogadoEsferaId                 1 non-null      object 
 11  orga

In [ ]:
df_licitacoes_idCompra = df_licitacoes_idCompra.drop([
      'orgaoSubrogadoCnpj',
      'orgaoSubrogadoRazaoSocial',
      'orgaoSubrogadoEsferaId',
      'orgaoSubrogadoPoderId',
      'unidadeSubrogadaCodigoUnidade',
      'unidadeSubrogadaNomeUnidade',
      'unidadeSubrogadaUfSigla',
      'unidade_subrogada_municipio_nome',
      'unidadeSubrogadaCodigoIbge',
      'orgaoEntidadeRazaoSocial',
      'orgaoEntidadeCnpj',
      'orgaoEntidadeEsferaId',
      'orgaoEntidadePoderId',
      'unidadeOrgaoNomeUnidade',
      'unidadeOrgaoCodigoIbge',
      'amparoLegalCodigoPncp',
      'amparoLegalNome',
      'amparoLegalDescricao',
      'dataAualizacaoPncp',
      'dataInclusaoPncp',
      'contratacaoExcluida'], axis = 1)

In [ ]:
df_licitacoes_idCompra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 31 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   idCompra                               1068 non-null   object 
 1   numeroControlePNCP                     1068 non-null   object 
 2   anoCompraPncp                          1068 non-null   int64  
 3   sequencialCompraPncp                   1068 non-null   int64  
 4   codigoOrgao                            1068 non-null   int64  
 5   unidadeOrgaoCodigoUnidade              1068 non-null   object 
 6   unidadeOrgaoUfSigla                    1068 non-null   object 
 7   unidadeOrgaoMunicipioNome              1068 non-null   object 
 8   numeroCompra                           1068 non-null   object 
 9   modalidadeIdPncp                       1068 non-null   int64  
 10  codigoModalidade                       1068 non-null   int64  
 11  moda

# Tratamento para modelo dos df e df_licitacoes_idCompra

In [ ]:
colunas_para_data_licitacoes_idCompra = ['dataPublicacaoPncp','dataAberturaPropostaPncp','dataEncerramentoPropostaPncp']

for coluna in colunas_para_data_licitacoes_idCompra:
    df_licitacoes_idCompra[coluna] = pd.to_datetime(df_licitacoes_idCompra[coluna], errors='coerce')

In [ ]:
print(df_licitacoes_idCompra['dataEncerramentoPropostaPncp'].max())
print(df_licitacoes_idCompra['dataEncerramentoPropostaPncp'].min())

2025-02-11 07:59:59
2022-02-11 07:59:59


In [ ]:
df_licitacoes_idCompra = df_licitacoes_idCompra[df_licitacoes_idCompra['anoCompraPncp'] != 2026]

In [ ]:
df_licitacoes_idCompra = df_licitacoes_idCompra[df_licitacoes_idCompra['modoDisputaNomePncp'] !='Não se aplica']

In [ ]:
df_licitacoes_idCompra_resumida = df_licitacoes_idCompra[[
    'idCompra',
    'anoCompraPncp',
    'unidadeOrgaoUfSigla',
    'modoDisputaNomePncp',
    'valorTotalEstimado',
    'valorTotalHomologado',
    'srp']]

df_licitacoes_idCompra_resumida.head()

,idCompra,anoCompraPncp,unidadeOrgaoUfSigla,modoDisputaNomePncp,valorTotalEstimado,valorTotalHomologado,srp
0,98767705900122024,2024,PR,Aberto,81066.53,6.395148e+04,False
2,98288705900222024,2024,AL,Aberto-Fechado,1046037.60,7.116000e+05,True
4,97400405900312024,2024,DF,Aberto,4723671.49,3.542754e+06,False
5,98777505900202024,2024,RS,Aberto,104280.00,7.198800e+04,False
7,92500305909932024,2024,SP,Aberto-Fechado,21658176.69,2.108600e+07,False


In [ ]:
df_licitacoes_idCompra_resumida['modoDisputaNomePncp'].unique()

array(['Aberto', 'Aberto-Fechado', 'Dispensa Com Disputa',
       'Fechado-Aberto', 'Fechado'], dtype=object)

In [ ]:
df_licitacoes_idCompra_resumida.info()

<class 'pandas.core.frame.DataFrame'>
Index: 525 entries, 0 to 1065
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   idCompra              525 non-null    object 
 1   anoCompraPncp         525 non-null    int64  
 2   unidadeOrgaoUfSigla   525 non-null    object 
 3   modoDisputaNomePncp   525 non-null    object 
 4   valorTotalEstimado    525 non-null    float64
 5   valorTotalHomologado  523 non-null    float64
 6   srp                   525 non-null    bool   
dtypes: bool(1), float64(2), int64(1), object(3)
memory usage: 29.2+ KB


In [ ]:
df_licitacoes_idCompra_resumida = df_licitacoes_idCompra_resumida.dropna()

In [ ]:
df_licitações_categorical_columns = [
    col for col in ['anoCompraPncp', 'unidadeOrgaoUfSigla', 'modoDisputaNomePncp', 'srp']
    if col in df_licitacoes_idCompra_resumida.columns
]

# Convertendo cada coluna individualmente para evitar o erro
for col in df_licitações_categorical_columns:
    df_licitacoes_idCompra_resumida[col] = df_licitacoes_idCompra_resumida[col].astype('category')

In [ ]:
df_licitações_int_columns = ['valorTotalEstimado', 'valorTotalHomologado']

df_licitacoes_idCompra_resumida[df_licitações_int_columns] = df_licitacoes_idCompra_resumida[df_licitações_int_columns].astype('int')

df_licitacoes_idCompra_resumida.info()

<class 'pandas.core.frame.DataFrame'>
Index: 523 entries, 0 to 1065
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   idCompra              523 non-null    object  
 1   anoCompraPncp         523 non-null    category
 2   unidadeOrgaoUfSigla   523 non-null    category
 3   modoDisputaNomePncp   523 non-null    category
 4   valorTotalEstimado    523 non-null    int64   
 5   valorTotalHomologado  523 non-null    int64   
 6   srp                   523 non-null    category
dtypes: category(4), int64(2), object(1)
memory usage: 20.1+ KB


In [ ]:
df_licitacoes_idCompra_resumida.duplicated().sum()

0

In [ ]:
df_licitacoes_idCompra_resumida = df_licitacoes_idCompra_resumida[df_licitacoes_idCompra_resumida['valorTotalHomologado'].notna()]
df_licitacoes_idCompra_resumida.isna().sum()

,0
idCompra,0
anoCompraPncp,0
unidadeOrgaoUfSigla,0
modoDisputaNomePncp,0
valorTotalEstimado,0
valorTotalHomologado,0
srp,0


In [ ]:
# ProfileReport(df)

In [ ]:
df = df.drop(
    ['codigoOrgao', 'nomeOrgao', 'modalidade','codigoUasg','nomeUasg', 'codigoMunicipio', 'municipio','estado',
     'dataHoraAtualizacaoCompra','dataHoraAtualizacaoCompra','nomeUnidadeMedida','dataHoraAtualizacaoItem','dataResultado',
     'dataHoraAtualizacaoUasg','percentualMaiorDesconto'
    ],
    axis=1)

In [ ]:
df.head()

,idCompra,idItemCompra,criterioJulgamento,numeroItemCompra,descricaoItem,codigoItemCatalogo,siglaUnidadeMedida,quantidade,precoUnitario,niFornecedor,nomeFornecedor,dataCompra
0,98122305000072022,787350,V,1,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,25852,PF,1.0,72000.00,12378206000139,CONTREINA - CONSULTORIA E TREINAMENTO EM SOFTW...,2022-05-10 03:00:00+00:00
1,29000205000822022,1662699,V,1,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,25852,UST,10.0,240710.52,11168199000188,HITSS DO BRASIL SERVICOS TECNOLOGICOS LTDA.,2022-10-24 03:00:00+00:00
2,98755305000372023,3520859,V,16,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,25852,UST,12.0,910.00,84785070000192,PRODASP INFORMATICA LTDA,2023-09-05 03:00:00+00:00
3,98755305000372023,3520864,V,21,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,25852,UST,12.0,637.04,84785070000192,PRODASP INFORMATICA LTDA,2023-09-05 03:00:00+00:00
4,98755305000372023,3520865,V,22,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,25852,UST,12.0,633.43,84785070000192,PRODASP INFORMATICA LTDA,2023-09-05 03:00:00+00:00


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6149 entries, 0 to 6589
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   idCompra            6149 non-null   object             
 1   idItemCompra        6149 non-null   int64              
 2   criterioJulgamento  6149 non-null   object             
 3   numeroItemCompra    6149 non-null   int64              
 4   descricaoItem       6149 non-null   object             
 5   codigoItemCatalogo  6149 non-null   int64              
 6   siglaUnidadeMedida  6147 non-null   object             
 7   quantidade          6149 non-null   float64            
 8   precoUnitario       6149 non-null   float64            
 9   niFornecedor        6149 non-null   object             
 10  nomeFornecedor      6149 non-null   object             
 11  dataCompra          6065 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(2), 

In [ ]:
df_resumido = df

df_resumido.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6149 entries, 0 to 6589
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   idCompra            6149 non-null   object             
 1   idItemCompra        6149 non-null   int64              
 2   criterioJulgamento  6149 non-null   object             
 3   numeroItemCompra    6149 non-null   int64              
 4   descricaoItem       6149 non-null   object             
 5   codigoItemCatalogo  6149 non-null   int64              
 6   siglaUnidadeMedida  6147 non-null   object             
 7   quantidade          6149 non-null   float64            
 8   precoUnitario       6149 non-null   float64            
 9   niFornecedor        6149 non-null   object             
 10  nomeFornecedor      6149 non-null   object             
 11  dataCompra          6065 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(2), 

In [ ]:
# df_resumido['descricaoItem'].unique()

In [ ]:
lista_drop_df_resumido = ['MENSURACAO DE SOFTWARE',
                          'SERVICOS HOSPEDAGEM DE SITIOS NA REDE MUNDIAL COMPUTADORES',
                          'SERVICOS DE HOSPEDAGEM DE APLICATIVOS E PROGRAMAS',
                          'SERVICOS DE HOSPEDAGEM DE DADOS',
                          'SERVICOS DE HOSPEDAGEM DE SISTEMAS',
                          'OUTROS SERVICOS DE HOSPEDAGEM EM  TECNOLOGIA DA INFORMACAO ECOMUNICACAO (TIC)',
                          'SERVICO DE HOSPEDAGEM DE SISTEMA DE CORREIO ELETRONICO',
                          'SERVICOS DE  GERENCIAMENTO  ELETRONICO DE DOCUMENTOS (GED) EWORKFLOW',
                          'SERVICOS ESPECIALIZADOS DE GEOPROCESSAMENTO',
                          'CONSULTORIA E ASSESSORIA - INTERNET',
                          'SERVICOS DE CONSULTORIA EM TECNOLOGIA DA INFORMACAO E COMUNICACAO (TIC)',
                          'SERVICOS DE CONSULTORIA EM  SEGURANCA DE TECNOLOGIA DA INFORMACAO E COMUNICACAO (TIC)',
                          'SERVIÇO DE TIC DE CARÁTER SECRETO OU RESERVADO']


# Identificar os idCompra que possuem pelo menos um dos itens na lista
ids_para_remover = df_resumido.loc[df_resumido['descricaoItem'].isin(lista_drop_df_resumido), 'idCompra'].unique()

# Remover todas as linhas que possuem esses idCompra
df_resumido = df_resumido[~df_resumido['idCompra'].isin(ids_para_remover)]

In [ ]:
df_resumido.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4064 entries, 0 to 5881
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   idCompra            4064 non-null   object             
 1   idItemCompra        4064 non-null   int64              
 2   criterioJulgamento  4064 non-null   object             
 3   numeroItemCompra    4064 non-null   int64              
 4   descricaoItem       4064 non-null   object             
 5   codigoItemCatalogo  4064 non-null   int64              
 6   siglaUnidadeMedida  4062 non-null   object             
 7   quantidade          4064 non-null   float64            
 8   precoUnitario       4064 non-null   float64            
 9   niFornecedor        4064 non-null   object             
 10  nomeFornecedor      4064 non-null   object             
 11  dataCompra          4007 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(2), 

In [ ]:
df_resumido['Num_max_itens'] = df_resumido['idCompra'].map(
    df_resumido.groupby('idCompra')['numeroItemCompra'].max())

df_resumido.head()

<ipython-input-319-c66f03870b95>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resumido['Num_max_itens'] = df_resumido['idCompra'].map(


,idCompra,idItemCompra,criterioJulgamento,numeroItemCompra,descricaoItem,codigoItemCatalogo,siglaUnidadeMedida,quantidade,precoUnitario,niFornecedor,nomeFornecedor,dataCompra,Num_max_itens
0,98122305000072022,787350,V,1,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,25852,PF,1.0,72000.00,12378206000139,CONTREINA - CONSULTORIA E TREINAMENTO EM SOFTW...,2022-05-10 03:00:00+00:00,1
1,29000205000822022,1662699,V,1,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,25852,UST,10.0,240710.52,11168199000188,HITSS DO BRASIL SERVICOS TECNOLOGICOS LTDA.,2022-10-24 03:00:00+00:00,1
2,98755305000372023,3520859,V,16,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,25852,UST,12.0,910.00,84785070000192,PRODASP INFORMATICA LTDA,2023-09-05 03:00:00+00:00,46
3,98755305000372023,3520864,V,21,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,25852,UST,12.0,637.04,84785070000192,PRODASP INFORMATICA LTDA,2023-09-05 03:00:00+00:00,46
4,98755305000372023,3520865,V,22,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,25852,UST,12.0,633.43,84785070000192,PRODASP INFORMATICA LTDA,2023-09-05 03:00:00+00:00,46


In [ ]:
df_resumido = df_resumido.drop(columns=['nomeFornecedor','idItemCompra','codigoItemCatalogo','niFornecedor','numeroItemCompra','precoUnitario','dataCompra'])

In [ ]:
df_resumido.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4064 entries, 0 to 5881
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   idCompra            4064 non-null   object 
 1   criterioJulgamento  4064 non-null   object 
 2   descricaoItem       4064 non-null   object 
 3   siglaUnidadeMedida  4062 non-null   object 
 4   quantidade          4064 non-null   float64
 5   Num_max_itens       4064 non-null   int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 222.2+ KB


In [ ]:
categorical_columns = ['criterioJulgamento','descricaoItem','siglaUnidadeMedida']
int_columns = ['quantidade','Num_max_itens']


df_resumido[categorical_columns] = df_resumido[categorical_columns].astype('category')
df_resumido[int_columns] = df_resumido[int_columns].astype('int')

df_resumido.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4064 entries, 0 to 5881
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   idCompra            4064 non-null   object  
 1   criterioJulgamento  4064 non-null   category
 2   descricaoItem       4064 non-null   category
 3   siglaUnidadeMedida  4062 non-null   category
 4   quantidade          4064 non-null   int64   
 5   Num_max_itens       4064 non-null   int64   
dtypes: category(3), int64(2), object(1)
memory usage: 140.2+ KB


In [ ]:
df_licitacoes_idCompra_resumida.info()

<class 'pandas.core.frame.DataFrame'>
Index: 523 entries, 0 to 1065
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   idCompra              523 non-null    object  
 1   anoCompraPncp         523 non-null    category
 2   unidadeOrgaoUfSigla   523 non-null    category
 3   modoDisputaNomePncp   523 non-null    category
 4   valorTotalEstimado    523 non-null    int64   
 5   valorTotalHomologado  523 non-null    int64   
 6   srp                   523 non-null    category
dtypes: category(4), int64(2), object(1)
memory usage: 20.1+ KB


In [ ]:
df_final = pd.merge(df_licitacoes_idCompra_resumida, df_resumido, on='idCompra', how='left')

df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846 entries, 0 to 845
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   idCompra              846 non-null    object  
 1   anoCompraPncp         846 non-null    category
 2   unidadeOrgaoUfSigla   846 non-null    category
 3   modoDisputaNomePncp   846 non-null    category
 4   valorTotalEstimado    846 non-null    int64   
 5   valorTotalHomologado  846 non-null    int64   
 6   srp                   846 non-null    category
 7   criterioJulgamento    605 non-null    category
 8   descricaoItem         605 non-null    category
 9   siglaUnidadeMedida    605 non-null    category
 10  quantidade            605 non-null    float64 
 11  Num_max_itens         605 non-null    float64 
dtypes: category(7), float64(2), int64(2), object(1)
memory usage: 42.0+ KB


In [ ]:
df_final = df_final.dropna()
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 605 entries, 1 to 776
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   idCompra              605 non-null    object  
 1   anoCompraPncp         605 non-null    category
 2   unidadeOrgaoUfSigla   605 non-null    category
 3   modoDisputaNomePncp   605 non-null    category
 4   valorTotalEstimado    605 non-null    int64   
 5   valorTotalHomologado  605 non-null    int64   
 6   srp                   605 non-null    category
 7   criterioJulgamento    605 non-null    category
 8   descricaoItem         605 non-null    category
 9   siglaUnidadeMedida    605 non-null    category
 10  quantidade            605 non-null    float64 
 11  Num_max_itens         605 non-null    float64 
dtypes: category(7), float64(2), int64(2), object(1)
memory usage: 35.5+ KB


In [ ]:
ajuste_int = ['quantidade','Num_max_itens']

df_final[ajuste_int] = df_final[ajuste_int].astype('int')

In [ ]:
df_features = df_final.copy()

In [ ]:
numerical_cols = df_features.select_dtypes('number').columns
Ordinal_ecoding_cols = ['unidadeOrgaoUfSigla','modoDisputaNomePncp','anoCompraPncp','srp','criterioJulgamento']
Onehot_encoding_cols = ['siglaUnidadeMedida','descricaoItem']

In [ ]:
# Inicializa o OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

# Ajusta e transforma as colunas categóricas
encoded_array = ordinal_encoder.fit_transform(df_features[Ordinal_ecoding_cols])

In [ ]:
encoded_array

array([[ 1.,  1.,  2.,  1.,  3.],
       [ 1.,  1.,  2.,  1.,  3.],
       [ 1.,  1.,  2.,  1.,  3.],
       ...,
       [ 6.,  1.,  2.,  0.,  3.],
       [15.,  2.,  2.,  0.,  1.],
       [20.,  2.,  2.,  0.,  1.]])

In [ ]:
# Cria um DataFrame com as colunas Ordinal Encoded
encoded_df_ordinal = pd.DataFrame(encoded_array, columns=Ordinal_ecoding_cols)

encoded_df_ordinal.reset_index(drop=True, inplace=True)

df_features = df_features.drop(columns=Ordinal_ecoding_cols)

df_features = pd.concat([df_features.reset_index(drop=True), encoded_df_ordinal], axis=1)

df_features.head()

,idCompra,valorTotalEstimado,valorTotalHomologado,descricaoItem,siglaUnidadeMedida,quantidade,Num_max_itens,unidadeOrgaoUfSigla,modoDisputaNomePncp,anoCompraPncp,srp,criterioJulgamento
0,98288705900222024,1046037,711600,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,,60,4,1.0,1.0,2.0,1.0,3.0
1,98288705900222024,1046037,711600,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,,60,4,1.0,1.0,2.0,1.0,3.0
2,98288705900222024,1046037,711600,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,,60,4,1.0,1.0,2.0,1.0,3.0
3,98288705900222024,1046037,711600,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,,60,4,1.0,1.0,2.0,1.0,3.0
4,97400405900312024,4723671,3542753,DESENVOLVIMENTO DE NOVO SOFTWARE - JAVA,,48,1,6.0,0.0,2.0,0.0,3.0


In [ ]:
# Criar um dicionário para armazenar os mapeamentos
ordinal_mappings = {}

# Percorrer as colunas categóricas e armazenar os mapeamentos
for col, categories in zip(Ordinal_ecoding_cols, ordinal_encoder.categories_):
    ordinal_mappings[col] = {category: code for code, category in enumerate(categories)}

# Exibir os mapeamentos
for col, mapping in ordinal_mappings.items():
    print(f"Coluna: {col}")
    print(mapping)
    print()

Coluna: unidadeOrgaoUfSigla
{'AC': 0, 'AL': 1, 'AM': 2, 'AP': 3, 'BA': 4, 'CE': 5, 'DF': 6, 'ES': 7, 'GO': 8, 'MA': 9, 'MG': 10, 'MS': 11, 'MT': 12, 'PA': 13, 'PB': 14, 'PE': 15, 'PI': 16, 'PR': 17, 'RJ': 18, 'RN': 19, 'RO': 20, 'RR': 21, 'RS': 22, 'SC': 23, 'SE': 24, 'SP': 25, 'TO': 26}

Coluna: modoDisputaNomePncp
{'Aberto': 0, 'Aberto-Fechado': 1, 'Dispensa Com Disputa': 2, 'Fechado': 3, 'Fechado-Aberto': 4}

Coluna: anoCompraPncp
{2022: 0, 2023: 1, 2024: 2, 2025: 3}

Coluna: srp
{0.0: 0, 1.0: 1}

Coluna: criterioJulgamento
{' ': 0, '1': 1, 'T': 2, 'V': 3}



In [ ]:
df_features = df_features[df_features['siglaUnidadeMedida']!='']

In [ ]:
if Onehot_encoding_cols:

    # Inicializa o OneHotEncoder
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

    # Ajusta e transforma as colunas categóricas
    encoded_array = encoder.fit_transform(df_features[Onehot_encoding_cols])

    # Cria um DataFrame com as novas colunas OneHotEncoded
    encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(Onehot_encoding_cols))

    # Reseta o índice para garantir a concatenação correta
    encoded_df.reset_index(drop=True, inplace=True)

    # Junta o DataFrame original com o DataFrame codificado
    df_features = pd.concat([df_features.reset_index(drop=True), encoded_df], axis=1)

    # Remove as colunas categóricas originais
    df_features = df_features.drop(columns=Onehot_encoding_cols)

# Exibir o DataFrame resultante
df_features.head()

,idCompra,valorTotalEstimado,valorTotalHomologado,quantidade,Num_max_itens,unidadeOrgaoUfSigla,modoDisputaNomePncp,anoCompraPncp,srp,criterioJulgamento,...,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - MAINFRAME,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - OUTRAS LINGUAGENS,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - PHP,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - PYTHON,"descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES)-FRAMEWORK.NET: C#,VISUAL BASIC,ASP,DELPHI, PASCAL",descricaoItem_QUALIDADE DE SOFTWARE,descricaoItem_SERVICO ESPECIALIZADO DE BUSINESS INTELIGENCE (BI),descricaoItem_SERVICOS DE INTEGRACAO DE SISTEMAS EM TECNOLOGIA DA INFORMAÇAO E COMUNICACAO (TIC),"descricaoItem_SERVICOS DE PESQUISA, ANALISE E DESENVOLVIMENTO EM TECNOLOGIA DA INFORMACAO E COMUNICACAO (TIC)",descricaoItem_SUSTENTACAO DE SOFTWARE
0,98777505900202024,104280,71988,12,1,22.0,0.0,2.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,92500305909932024,21658176,21086000,1,9,25.0,1.0,2.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,92500305909932024,21658176,21086000,1,9,25.0,1.0,2.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,92500305909932024,21658176,21086000,3000,9,25.0,1.0,2.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,92500305909932024,21658176,21086000,1,9,25.0,1.0,2.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 40 columns):
 #   Column                                                                                                                                  Non-Null Count  Dtype  
---  ------                                                                                                                                  --------------  -----  
 0   idCompra                                                                                                                                419 non-null    object 
 1   valorTotalEstimado                                                                                                                      419 non-null    int64  
 2   valorTotalHomologado                                                                                                                    419 non-null    int64  
 3   quantidade                                                       

In [ ]:
colunas_unidade_medida = [col for col in df_features.columns if col.startswith("siglaUnidadeMedida_")]

colunas_descrição_item = [col for col in df_features.columns if col.startswith("descricaoItem_")]

In [ ]:
df_features[colunas_unidade_medida] = df_features[colunas_unidade_medida].mul(df_features['quantidade'], axis=0)

df_features.head()

,idCompra,valorTotalEstimado,valorTotalHomologado,quantidade,Num_max_itens,unidadeOrgaoUfSigla,modoDisputaNomePncp,anoCompraPncp,srp,criterioJulgamento,...,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - MAINFRAME,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - OUTRAS LINGUAGENS,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - PHP,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - PYTHON,"descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES)-FRAMEWORK.NET: C#,VISUAL BASIC,ASP,DELPHI, PASCAL",descricaoItem_QUALIDADE DE SOFTWARE,descricaoItem_SERVICO ESPECIALIZADO DE BUSINESS INTELIGENCE (BI),descricaoItem_SERVICOS DE INTEGRACAO DE SISTEMAS EM TECNOLOGIA DA INFORMAÇAO E COMUNICACAO (TIC),"descricaoItem_SERVICOS DE PESQUISA, ANALISE E DESENVOLVIMENTO EM TECNOLOGIA DA INFORMACAO E COMUNICACAO (TIC)",descricaoItem_SUSTENTACAO DE SOFTWARE
0,98777505900202024,104280,71988,12,1,22.0,0.0,2.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,92500305909932024,21658176,21086000,1,9,25.0,1.0,2.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,92500305909932024,21658176,21086000,1,9,25.0,1.0,2.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,92500305909932024,21658176,21086000,3000,9,25.0,1.0,2.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,92500305909932024,21658176,21086000,1,9,25.0,1.0,2.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_features = df_features.drop(columns=['quantidade'])

df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 39 columns):
 #   Column                                                                                                                                  Non-Null Count  Dtype  
---  ------                                                                                                                                  --------------  -----  
 0   idCompra                                                                                                                                419 non-null    object 
 1   valorTotalEstimado                                                                                                                      419 non-null    int64  
 2   valorTotalHomologado                                                                                                                    419 non-null    int64  
 3   Num_max_itens                                                    

In [ ]:
df_features_pivot = df_features.groupby('idCompra', as_index=False).agg(
    {col: 'sum' for col in colunas_unidade_medida + colunas_descrição_item})

df_features_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 31 columns):
 #   Column                                                                                                                                  Non-Null Count  Dtype  
---  ------                                                                                                                                  --------------  -----  
 0   idCompra                                                                                                                                198 non-null    object 
 1   siglaUnidadeMedida_H                                                                                                                    198 non-null    float64
 2   siglaUnidadeMedida_HST                                                                                                                  198 non-null    float64
 3   siglaUnidadeMedida_PF                                            

In [ ]:
df_features_pivot.describe()

,siglaUnidadeMedida_H,siglaUnidadeMedida_HST,siglaUnidadeMedida_PF,siglaUnidadeMedida_PFM,siglaUnidadeMedida_PFS,siglaUnidadeMedida_PPA,siglaUnidadeMedida_SPT,siglaUnidadeMedida_UN,siglaUnidadeMedida_UST,descricaoItem_DESENVOLVIMENTO DE NOVO SOFTWARE - DISPOSITIVOS MOVEIS,...,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - MAINFRAME,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - OUTRAS LINGUAGENS,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - PHP,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - PYTHON,"descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES)-FRAMEWORK.NET: C#,VISUAL BASIC,ASP,DELPHI, PASCAL",descricaoItem_QUALIDADE DE SOFTWARE,descricaoItem_SERVICO ESPECIALIZADO DE BUSINESS INTELIGENCE (BI),descricaoItem_SERVICOS DE INTEGRACAO DE SISTEMAS EM TECNOLOGIA DA INFORMAÇAO E COMUNICACAO (TIC),"descricaoItem_SERVICOS DE PESQUISA, ANALISE E DESENVOLVIMENTO EM TECNOLOGIA DA INFORMACAO E COMUNICACAO (TIC)",descricaoItem_SUSTENTACAO DE SOFTWARE
count,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,...,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000
mean,105.808081,217.979798,158.767677,0.803030,0.060606,50.666667,0.005051,93.550505,3675.666667,0.055556,...,0.010101,0.101010,0.015152,0.005051,0.025253,0.010101,0.035354,0.429293,0.101010,0.222222
std,1292.276590,1418.418892,1060.373115,4.534872,0.852803,710.659606,0.071067,1111.523021,32941.212316,0.392757,...,0.100248,0.542507,0.122465,0.071067,0.157289,0.100248,0.185140,1.822741,0.675833,0.754756
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,17950.000000,16128.000000,12500.000000,36.000000,12.000000,10000.000000,1.000000,15608.000000,445000.000000,5.000000,...,1.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,20.000000,9.000000,9.000000


In [ ]:
df_features_menos_pivot =  df_features.drop(columns=colunas_unidade_medida + colunas_descrição_item)

df_features_final = pd.merge(df_features_menos_pivot, df_features_pivot, on='idCompra', how='inner')

In [ ]:
df_features_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 39 columns):
 #   Column                                                                                                                                  Non-Null Count  Dtype  
---  ------                                                                                                                                  --------------  -----  
 0   idCompra                                                                                                                                419 non-null    object 
 1   valorTotalEstimado                                                                                                                      419 non-null    int64  
 2   valorTotalHomologado                                                                                                                    419 non-null    int64  
 3   Num_max_itens                                                    

In [ ]:
df_features_final.duplicated().sum()

221

In [ ]:
df_features_final = df_features_final.drop_duplicates()

In [ ]:
df_features_final = df_features_final.drop(columns=['idCompra'])

In [ ]:
df_features_final.corr()[['valorTotalHomologado']].sort_values(by='valorTotalHomologado', ascending=False)

,valorTotalHomologado
valorTotalHomologado,1.000000
valorTotalEstimado,0.915366
siglaUnidadeMedida_UN,0.313301
srp,0.306238
descricaoItem_SUSTENTACAO DE SOFTWARE,0.257041
Num_max_itens,0.231710
criterioJulgamento,0.171996
siglaUnidadeMedida_PPA,0.097570
descricaoItem_DESENVOLVIMENTO DE NOVO SOFTWARE - MAINFRAME,0.097521
descricaoItem_DESENVOLVIMENTO DE NOVO SOFTWARE - PHP,0.091879


# Modelo de previsão de preço final a partir do valor homologado, itens e quantidades.


In [ ]:
X = df_features_final.drop(columns=['valorTotalHomologado','modoDisputaNomePncp'])
y = df_features_final['valorTotalHomologado']

# Dividir os dados em treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
list_r_2 = list()
list_mse = list()

In [ ]:
# Criar o modelo de regressão regressão por arvoré
model_tree = DecisionTreeRegressor()

# Treinar o modelo nos dados de treino
model_tree.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_tree = model_tree.predict(X_test)

# Avaliar a performance do modelo
mse_tree = mean_squared_error(y_test, y_pred_tree)
r2_tree = r2_score(y_test, y_pred_tree)

list_r_2.append(r2_tree)
list_mse.append(mse_tree)

print(f"Mean Squared Error (MSE): {mse_tree:.2f}")
print(f"R² Score: {r2_tree:.2f}")

Mean Squared Error (MSE): 6906011423388.30
R² Score: 0.70


In [ ]:
# Criar o modelo de regressão XGBoost
model_XGB = XGBRegressor()

# Treinar o modelo nos dados de treino
model_XGB.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_XGB = model_XGB.predict(X_test)

# Avaliar a performance do modelo
mse_XGB = mean_squared_error(y_test, y_pred_XGB)
r2_XGB = r2_score(y_test, y_pred_XGB)

list_r_2.append(r2_XGB)
list_mse.append(mse_XGB)

print(f"Mean Squared Error (MSE): {mse_XGB:.4f}")
print(f"R² Score: {r2_XGB:.4f}")

Mean Squared Error (MSE): 2988317081600.0000
R² Score: 0.8686


In [ ]:
# Criar o modelo de regressão CatBoost
model_cb = CatBoostRegressor(learning_rate=0.1, iterations=1000)

# Treinar o modelo nos dados de treino
model_cb.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_cb = model_cb.predict(X_test)

# Avaliar a performance do modelo
mse_cb = mean_squared_error(y_test, y_pred_cb)
r2_cb = r2_score(y_test, y_pred_cb)

list_r_2.append(r2_cb)
list_mse.append(mse_cb)

print(f"Mean Squared Error (MSE): {mse_cb:.4f}")
print(f"R² Score: {r2_cb:.4f}")

0:	learn: 10086833.1556345	total: 1.27ms	remaining: 1.27s
1:	learn: 9612778.8559232	total: 2.18ms	remaining: 1.09s
2:	learn: 9197477.9094401	total: 3.02ms	remaining: 1s
3:	learn: 8920761.9651563	total: 3.67ms	remaining: 914ms
4:	learn: 8613904.8590783	total: 4.63ms	remaining: 921ms
5:	learn: 8216092.2603024	total: 5.59ms	remaining: 927ms
6:	learn: 7932678.3420946	total: 6.55ms	remaining: 929ms
7:	learn: 7626255.1543026	total: 7.5ms	remaining: 930ms
8:	learn: 7378904.6627614	total: 8.43ms	remaining: 928ms
9:	learn: 7077296.7403844	total: 9.35ms	remaining: 926ms
10:	learn: 6838754.4558996	total: 10.2ms	remaining: 914ms
11:	learn: 6577502.2284545	total: 10.7ms	remaining: 881ms
12:	learn: 6328150.1521309	total: 11.6ms	remaining: 880ms
13:	learn: 6166864.9596334	total: 12.2ms	remaining: 861ms
14:	learn: 5930079.5320355	total: 12.8ms	remaining: 838ms
15:	learn: 5767108.7062295	total: 13.7ms	remaining: 841ms
16:	learn: 5589872.5060170	total: 14.6ms	remaining: 843ms
17:	learn: 5404506.6989977	

In [ ]:
modelos = ['Decision Tree', 'XGBoost', 'Catboost']

# Create the DataFrame
df_results = pd.DataFrame({'Modelos': modelos, 'R^2': list_r_2, 'MSE': list_mse})

# Display the DataFrame
df_results

,Modelos,R^2,MSE
0,Decision Tree,0.696274,6.906011e+12
1,XGBoost,0.868574,2.988317e+12
2,Catboost,0.841463,3.604748e+12


# GridSearch

In [ ]:
# Definir o modelo de regressão CatBoost (sem treinar ainda)
model_cb_max = CatBoostRegressor(verbose=0)  # Desativa logs para não poluir a saída

# Definir os hiperparâmetros a serem testados
param_grid_cb = {
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8],
    'iterations': [500, 1000],
    'l2_leaf_reg': [1, 3, 5],
    'border_count': [32, 64]  # Número de divisões para variáveis categóricas
}

In [ ]:
# Criar o modelo base XGBoost (sem treinar ainda)
model_XGB_max = XGBRegressor(objective='reg:squarederror')

# Definir os hiperparâmetros a serem testados
param_grid_XGB = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 6, 9],
    'subsample': [0.7, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.9, 1.0],
    'reg_lambda': [1, 3, 5]
}

In [ ]:
# Criar o modelo base Decision Tree
model_tree_max = DecisionTreeRegressor()

# Definir os hiperparâmetros a serem testados
param_grid_tree = {
    'max_depth': [3, 5, 10, None],  # Profundidade máxima da árvore
    'min_samples_split': [2, 5, 10],  # Número mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 5, 10],  # Número mínimo de amostras em uma folha
    'max_features': ['auto', 'sqrt', 'log2', None]  # Número máximo de features a considerar por divisão
}

In [ ]:
a = df_results.loc[df_results['R^2'].idxmax()]
print(a['Modelos'])

if a['Modelos'] == 'Decision Tree':
  param_grid = param_grid_tree
  model_ = model_tree_max

elif a['Modelos'] == 'XGBoost':
  param_grid = param_grid_XGB
  model_ = model_XGB_max

else:
  param_grid = param_grid_cb
  model_ = model_cb_max

XGBoost


In [ ]:
# Configurar o GridSearchCV para maximizar o MSE (minimizar o erro)
grid_search = GridSearchCV(model_, param_grid, scoring='neg_mean_squared_error', cv=3, n_jobs=10)

# Executar o GridSearchCV
grid_search.fit(X_train, y_train)

# Melhor combinação de hiperparâmetros
best_params = grid_search.best_params_
print(f"Melhores hiperparâmetros: {best_params}")

Melhores hiperparâmetros: {'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'reg_lambda': 1, 'subsample': 0.7}


In [ ]:
# Criar o modelo com os melhores parâmetros
if a['Modelos'] == 'Decision Tree':
  best_model_ = DecisionTreeRegressor(**best_params)

elif a['Modelos'] == 'XGBoost':
  best_model_ = XGBRegressor(**best_params, objective='reg:squarederror')

else:
  best_model_ = CatBoostRegressor(**best_params, verbose=0)

# Treinar o modelo nos dados de treino
best_model_.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_cb = best_model_.predict(X_test)

# Avaliar a performance do modelo otimizado
mse_cb = mean_squared_error(y_test, y_pred_cb)
r2_cb = r2_score(y_test, y_pred_cb)

print(f"Mean Squared Error (MSE) após otimização: {mse_cb:.4f}")
print(f"R² Score após otimização: {r2_cb:.4f}")

Mean Squared Error (MSE) após otimização: 3511101423616.0000
R² Score após otimização: 0.8456


In [ ]:
def best_model_pred (x):
  y_pred_best = best_model_.predict(x)
  return y_pred_best

# Teste com licitações Reais

In [ ]:
df_teste = pd.read_excel('/content/Base_teste_final.xlsx', sheet_name='Base_teste_Traduzida')

In [ ]:
df_teste.head()

,ID,Name,UASG,Número do Pregão,valorTotalEstimado,valorTotalHomologado,Num_max_itens,unidadeOrgaoUfSigla,modoDisputaNomePncp,anoCompraPncp,...,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - MAINFRAME,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - OUTRAS LINGUAGENS,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - PHP,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - PYTHON,"descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES)-FRAMEWORK.NET: C#,VISUAL BASIC,ASP,DELPHI, PASCAL",descricaoItem_QUALIDADE DE SOFTWARE,descricaoItem_SERVICO ESPECIALIZADO DE BUSINESS INTELIGENCE (BI),descricaoItem_SERVICOS DE INTEGRACAO DE SISTEMAS EM TECNOLOGIA DA INFORMAÇAO E COMUNICACAO (TIC),"descricaoItem_SERVICOS DE PESQUISA, ANALISE E DESENVOLVIMENTO EM TECNOLOGIA DA INFORMACAO E COMUNICACAO (TIC)",descricaoItem_SUSTENTACAO DE SOFTWARE
0,925057/90013/2024,SEHAB,925057,90013/2024,25549018.48,23890000,1,25,0,2,...,0,0,1,0,0,0,0,0,0,0
1,70005/90003/2025,TRIBUNAL REGIONAL ELEITORAL DO MARANHÃO,70005,90003/2025,1847616.00,0,1,9,1,3,...,0,0,0,0,0,0,0,0,0,1
2,114702/90001/2025,Fundação Escola Nacional de Adm. Pública,114702,90001/2025,7894269.47,4892000,6,6,0,3,...,0,0,0,0,0,0,0,0,0,0
3,926415/90003/2025,TRIBUNAL DE JUSTIÇA ESTADO DO PARANÁ,926415,90003/2025,33767640.00,16277750,2,17,0,3,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_teste_features = df_teste.set_index('ID').drop(columns=['Name','UASG','Número do Pregão','valorTotalHomologado','modoDisputaNomePncp'])
df_teste_features.head()

,valorTotalEstimado,Num_max_itens,unidadeOrgaoUfSigla,anoCompraPncp,srp,criterioJulgamento,siglaUnidadeMedida_H,siglaUnidadeMedida_HST,siglaUnidadeMedida_PF,siglaUnidadeMedida_PFM,...,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - MAINFRAME,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - OUTRAS LINGUAGENS,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - PHP,descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES) - PYTHON,"descricaoItem_MANUTENCAO EVOLUTIVA DE SOFTWARE (ACRESCIMO DE NOVAS FUNCIONALIDADES)-FRAMEWORK.NET: C#,VISUAL BASIC,ASP,DELPHI, PASCAL",descricaoItem_QUALIDADE DE SOFTWARE,descricaoItem_SERVICO ESPECIALIZADO DE BUSINESS INTELIGENCE (BI),descricaoItem_SERVICOS DE INTEGRACAO DE SISTEMAS EM TECNOLOGIA DA INFORMAÇAO E COMUNICACAO (TIC),"descricaoItem_SERVICOS DE PESQUISA, ANALISE E DESENVOLVIMENTO EM TECNOLOGIA DA INFORMACAO E COMUNICACAO (TIC)",descricaoItem_SUSTENTACAO DE SOFTWARE
ID,,,,,,,,,,,,,,,,,,,,,
925057/90013/2024,25549018.48,1,25,2,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
70005/90003/2025,1847616.00,1,9,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
114702/90001/2025,7894269.47,6,6,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
926415/90003/2025,33767640.00,2,17,3,0,0,0,0,17000,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_teste_features = df_teste_features.astype(float)

In [ ]:
y_teste_aletorios = best_model_pred (df_teste_features)

df_teste_resultados = pd.DataFrame(y_teste_aletorios)

In [ ]:
df_teste_final = pd.DataFrame()

df_teste_final['resultados'] = df_teste_resultados
df_teste_final['Valor_estimado'] = df_teste['valorTotalEstimado']
df_teste_final['%_desconto'] = ((df_teste_final['resultados']/df_teste['valorTotalEstimado'])-1)*100
df_teste_final['valorTotalHomologado'] = df_teste['valorTotalHomologado']
df_teste_final['%_desconto_homologado'] = ((df_teste['valorTotalHomologado']/df_teste['valorTotalEstimado'])-1)*100
df_teste_final['Erro_absoluto'] = abs(df_teste_final['resultados'] - df_teste['valorTotalHomologado'])

In [ ]:
df_teste_final['ID'] = df_teste['ID']

df_teste_final = df_teste_final.set_index('ID')

In [ ]:
df_teste_final = df_teste_final.astype(float).round(2)

In [ ]:
df_teste_final.head()

,resultados,Valor_estimado,%_desconto,valorTotalHomologado,%_desconto_homologado,Erro_absoluto
ID,,,,,,
925057/90013/2024,20680924.0,25549018.48,-19.05,23890000.0,-6.49,3209076.0
70005/90003/2025,564275.0,1847616.00,-69.46,0.0,-100.00,564275.0
114702/90001/2025,5622017.5,7894269.47,-28.78,4892000.0,-38.03,730017.5
926415/90003/2025,20441088.0,33767640.00,-39.47,16277750.0,-51.79,4163338.0
